<a href="https://colab.research.google.com/github/iskra3138/GluonTS/blob/master/Custom_DeepFactor_YJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install --upgrade mxnet==1.6 gluonts

     |████████████████████████████████| 68.7MB 62kB/s 
     |████████████████████████████████| 419kB 52.2MB/s 
     |████████████████████████████████| 194kB 50.3MB/s 
     |████████████████████████████████| 7.4MB 46.9MB/s 
  Created wheel for ujson: filename=ujson-1.35-cp36-cp36m-linux_x86_64.whl size=68011 sha256=27f1eeb97f4b0dc1840660044f318019953436a31b09de20d87392eaa5ad74eb
  Stored in directory: /root/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
Successfully built ujson
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [0]:
# Third-party imports
%matplotlib inline
import mxnet as mx
from mxnet import gluon
#import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt
import json
#import os

In [0]:
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
#from gluonts.dataset.util import to_pandas

In [0]:
dataset = get_dataset('electricity', regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/electricity/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/electricity/test/data.json


In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.trainer.html
deepfactor_trainer=Trainer(
    ctx = 'cpu',
    epochs=100, # (default: 100).
    batch_size=32,# (default: 32).
    num_batches_per_epoch=50, # (default: 50).
    learning_rate=0.001, # (default:  10^−3 ).
    learning_rate_decay_factor=0.5, # (default: 0.5).
    patience = 10, # (default: 10).
    minimum_learning_rate=5e-05, # (default:  5⋅10^−5 ).
    clip_gradient = 10.0, # (default: 10).
    weight_decay=1e-08, #  (default  10^−8 ).
    init='xavier', #  (default: “xavier”).
    hybridize=True, 
    )

In [0]:
# change only context_length
DF_estimator = DeepFactorEstimator(
    freq=dataset.metadata.freq, # Time series frequency
    prediction_length=dataset.metadata.prediction_length, # Prediction length.
    num_hidden_global=50, # (default: 50).
    num_layers_global=1, # (default: 1).
    num_factors=10, # (default: 10).
    num_hidden_local=5, # (default: 5).
    num_layers_local=1, # (default: 1).
    cell_type='lstm', # (default: ‘lstm’).
    trainer = deepfactor_trainer, # (default: Trainer()).
    context_length=168, # (default: None, in which case context_length = prediction_length).
    num_parallel_samples=100, # (default: 100).
    cardinality=[int(dataset.metadata.feat_static_cat[0].cardinality)], # (default: list([1]).
    embedding_dimension=10, # (default: 10).
    #distr_output=StudentTOutput(), # (default: StudentTOutput()).
)

In [0]:
DF_predictor = DF_estimator.train(dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:11<00:00,  4.43it/s, epoch=100/100, avg_epoch_loss=6.77e+3]


In [0]:
# Evaluation for all test data
forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset.test, 
    predictor=DF_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])

In [0]:
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 2247/2247 [00:00<00:00, 45801.63it/s]


{
    "MSE": 18876499.804294366,
    "abs_error": 31932658.630607605,
    "abs_target_sum": 128632956.0,
    "abs_target_mean": 2385.272140631948,
    "seasonal_error": 189.4933819611677,
    "MASE": 2.5504296674374847,
    "MAPE": 0.3804413659421475,
    "sMAPE": 0.2775985060632508,
    "OWA": NaN,
    "MSIS": 71.7191248409653,
    "QuantileLoss[0.1]": 38076520.64631615,
    "Coverage[0.1]": 0.40097908322207443,
    "QuantileLoss[0.5]": 31932658.567898393,
    "Coverage[0.5]": 0.5643635959056528,
    "QuantileLoss[0.9]": 22974524.90050087,
    "Coverage[0.9]": 0.693072244474113,
    "RMSE": 4344.709403895083,
    "NRMSE": 1.8214732524163917,
    "ND": 0.24824632523105203,
    "wQuantileLoss[0.1]": 0.29600906198809696,
    "wQuantileLoss[0.5]": 0.248246324743547,
    "wQuantileLoss[0.9]": 0.17860527826555483,
    "mean_wQuantileLoss": 0.24095355499906626,
    "MAE_Coverage": 0.19075681155120475
}


In [0]:
#@title Define Custom Estimator
# Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License").
# You may not use this file except in compliance with the License.
# A copy of the License is located at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# or in the "license" file accompanying this file. This file is distributed
# on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
# express or implied. See the License for the specific language governing
# permissions and limitations under the License.

# Standard library imports
from typing import List, Optional

# First-party imports
from gluonts import transform
from gluonts.block.feature import FeatureEmbedder
from gluonts.core.component import validated
from gluonts.dataset.field_names import FieldName
from gluonts.distribution import DistributionOutput, StudentTOutput

from gluonts.model.deep_factor.RNNModel import RNNModel
from gluonts.model.deep_factor._network import (
    DeepFactorTrainingNetwork,
    DeepFactorPredictionNetwork,
)
from gluonts.model.estimator import GluonEstimator
from gluonts.model.predictor import Predictor, RepresentableBlockPredictor
from gluonts.time_feature import time_features_from_frequency_str
from gluonts.trainer import Trainer
from gluonts.transform import (
    AddTimeFeatures,
    AsNumpyArray,
    Chain,
    SetFieldIfNotPresent,
    TestSplitSampler,
    Transformation,
)


# Third-party imports


class custom_DeepFactorEstimator(GluonEstimator):
    r"""
    DeepFactorEstimator is an implementation of the 2019 ICML paper "Deep Factors for Forecasting"
    https://arxiv.org/abs/1905.12417.  It uses a global RNN model to learn patterns across multiple related time series
    and an arbitrary local model to model the time series on a per time series basis.  In the current implementation,
    the local model is a RNN (DF-RNN).

    Parameters
    ----------
    freq
        Time series frequency.
    prediction_length
        Prediction length.
    num_hidden_global
        Number of units per hidden layer for the global RNN model (default: 50).
    num_layers_global
        Number of hidden layers for the global RNN model (default: 1).
    num_factors
        Number of global factors (default: 10).
    num_hidden_local
        Number of units per hidden layer for the local RNN model (default: 5).
    num_layers_local
        Number of hidden layers for the global local model (default: 1).
    cell_type
        Type of recurrent cells to use (available: 'lstm' or 'gru';
        default: 'lstm').
    trainer
        Trainer object to be used (default: Trainer()).
    context_length
        Training length (default: None, in which case context_length = prediction_length).
    num_parallel_samples
        Number of evaluation samples per time series to increase parallelism during inference.
        This is a model optimization that does not affect the accuracy (default: 100).
    cardinality
        List consisting of the number of time series (default: list([1]).
    embedding_dimension
        Dimension of the embeddings for categorical features (the same
        dimension is used for all embeddings, default: 10).
    distr_output
        Distribution to use to evaluate observations and sample predictions
        (default: StudentTOutput()).
    """

    @validated()
    def __init__(
        self,
        freq: str,
        prediction_length: int,
        num_hidden_global: int = 50,
        num_layers_global: int = 1,
        num_factors: int = 10,
        num_hidden_local: int = 5,
        num_layers_local: int = 1,
        cell_type: str = "lstm",
        trainer: Trainer = Trainer(),
        context_length: Optional[int] = None,
        num_parallel_samples: int = 100,
        cardinality: List[int] = list([1]),
        embedding_dimension: int = 10,
        distr_output: DistributionOutput = StudentTOutput(),
    ) -> None:
        super().__init__(trainer=trainer)

        assert (
            prediction_length > 0
        ), "The value of `prediction_length` should be > 0"
        assert (
            context_length is None or context_length > 0
        ), "The value of `context_length` should be > 0"
        assert num_layers_global > 0, "The value of `num_layers` should be > 0"
        assert num_hidden_global > 0, "The value of `num_hidden` should be > 0"
        assert num_factors > 0, "The value of `num_factors` should be > 0"
        assert (
            num_hidden_local > 0
        ), "The value of `num_hidden_local` should be > 0"
        assert (
            num_layers_local > 0
        ), "The value of `num_layers_local` should be > 0"
        assert all(
            [c > 0 for c in cardinality]
        ), "Elements of `cardinality` should be > 0"
        assert (
            embedding_dimension > 0
        ), "The value of `embedding_dimension` should be > 0"
        assert (
            num_parallel_samples > 0
        ), "The value of `num_parallel_samples` should be > 0"

        self.freq = freq
        self.context_length = (
            context_length if context_length is not None else prediction_length
        )
        self.prediction_length = prediction_length
        self.distr_output = distr_output
        self.num_parallel_samples = num_parallel_samples
        self.cardinality = cardinality
        self.embedding_dimensions = [embedding_dimension for _ in cardinality]

        self.global_model = RNNModel(
            mode=cell_type,
            num_hidden=num_hidden_global,
            num_layers=num_layers_global,
            num_output=num_factors,
        )

        # TODO: Allow the local model to be defined as an arbitrary local model, e.g. DF-GP and DF-LDS
        self.local_model = RNNModel(
            mode=cell_type,
            num_hidden=num_hidden_local,
            num_layers=num_layers_local,
            num_output=1,
        )

    def create_transformation(self) -> Transformation:
        return Chain(
            trans=[
                AsNumpyArray(field=FieldName.TARGET, expected_ndim=1),
                AddObservedValuesIndicator(
                    target_field=FieldName.TARGET,
                    output_field=FieldName.OBSERVED_VALUES,
                    dummy_value=self.distr_output.value_in_support,
                    dtype=self.dtype,
                ),
                AddTimeFeatures(
                    start_field=FieldName.START,
                    target_field=FieldName.TARGET,
                    output_field=FieldName.FEAT_TIME,
                    time_features=time_features_from_frequency_str(self.freq),
                    pred_length=self.prediction_length,
                ),
                AddAgeFeature(
                    target_field=FieldName.TARGET,
                    output_field=FieldName.FEAT_AGE,
                    pred_length=self.prediction_length,
                    log_scale=True,
                    dtype=self.dtype,
                ),
                VstackFeatures(
                    output_field=FieldName.FEAT_TIME,
                    input_fields=[FieldName.FEAT_TIME, FieldName.FEAT_AGE]
                ),
                SetFieldIfNotPresent(
                    field=FieldName.FEAT_STATIC_CAT, value=[0.0]
                ),
                AsNumpyArray(field=FieldName.FEAT_STATIC_CAT, expected_ndim=1),
                transform.InstanceSplitter(
                    target_field=FieldName.TARGET,
                    is_pad_field=FieldName.IS_PAD,
                    start_field=FieldName.START,
                    forecast_start_field=FieldName.FORECAST_START,
                    train_sampler=TestSplitSampler(),
                    time_series_fields=[
                        FieldName.FEAT_TIME,
                        FieldName.OBSERVED_VALUES,
                    ],
                    past_length=self.context_length,
                    future_length=self.prediction_length,
                ),
            ]
        )

    def create_training_network(self) -> DeepFactorTrainingNetwork:
        return DeepFactorTrainingNetwork(
            embedder=FeatureEmbedder(
                cardinalities=self.cardinality,
                embedding_dims=self.embedding_dimensions,
            ),
            global_model=self.global_model,
            local_model=self.local_model,
        )

    def create_predictor(
        self,
        transformation: Transformation,
        trained_network: DeepFactorTrainingNetwork,
    ) -> Predictor:
        prediction_net = DeepFactorPredictionNetwork(
            embedder=trained_network.embedder,
            global_model=trained_network.global_model,
            local_model=trained_network.local_model,
            prediction_len=self.prediction_length,
            num_parallel_samples=self.num_parallel_samples,
            params=trained_network.collect_params(),
        )

        return RepresentableBlockPredictor(
            input_transform=transformation,
            prediction_net=prediction_net,
            batch_size=self.trainer.batch_size,
            freq=self.freq,
            prediction_length=self.prediction_length,
            ctx=self.trainer.ctx,
        )

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deep_factor.html
DF_estimator = custom_DeepFactorEstimator(
    freq=dataset.metadata.freq, # Time series frequency
    prediction_length=dataset.metadata.prediction_length, # Prediction length.
    num_hidden_global=50, # (default: 50).
    num_layers_global=1, # (default: 1).
    num_factors=10, # (default: 10).
    num_hidden_local=5, # (default: 5).
    num_layers_local=1, # (default: 1).
    cell_type='lstm', # (default: ‘lstm’).
    trainer = deepfactor_trainer, # (default: Trainer()).
    context_length=168, # (default: None, in which case context_length = prediction_length).
    num_parallel_samples=100, # (default: 100).
    cardinality=[int(dataset.metadata.feat_static_cat[0].cardinality)], # (default: list([1]).
    embedding_dimension=10, # (default: 10).
    #distr_output=StudentTOutput(), # (default: StudentTOutput()).
)

In [0]:
DF_predictor = DF_estimator.train(dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:14<00:00,  3.37it/s, epoch=100/100, avg_epoch_loss=8.19e+3]


In [0]:
# Evaluation for all test data
forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset.test, 
    predictor=DF_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 2247/2247 [00:00<00:00, 39655.98it/s]


{
    "MSE": 21318475.439580247,
    "abs_error": 31645725.93480301,
    "abs_target_sum": 128632956.0,
    "abs_target_mean": 2385.272140631948,
    "seasonal_error": 189.4933819611677,
    "MASE": 2.4330016020356733,
    "MAPE": 0.33228281572004065,
    "sMAPE": 0.26993763528768305,
    "OWA": NaN,
    "MSIS": 67.34964659011494,
    "QuantileLoss[0.1]": 22104872.01756821,
    "Coverage[0.1]": 0.26717104287197724,
    "QuantileLoss[0.5]": 31645726.060558725,
    "Coverage[0.5]": 0.41492360183948973,
    "QuantileLoss[0.9]": 38415444.580503464,
    "Coverage[0.9]": 0.5692404687731789,
    "RMSE": 4617.193459189278,
    "NRMSE": 1.9357092972905015,
    "ND": 0.24601569394707068,
    "wQuantileLoss[0.1]": 0.1718445467238443,
    "wQuantileLoss[0.5]": 0.24601569492470285,
    "wQuantileLoss[0.9]": 0.2986438761502415,
    "mean_wQuantileLoss": 0.23883470593292958,
    "MAE_Coverage": 0.19433565741976955
}
